In [1]:
import numpy as np
import keras
import json

Using TensorFlow backend.
C:\Users\Lenovo\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Lenovo\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Lenovo\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Lenovo\anaconda3\lib\site-packages\tensorflow\python\framework\dtype

In [2]:
SINGLE_FILE_DATASET = "C:/Users/Lenovo/Desktop/New folder/python/Song Generation/file_dataset"
SEQUENCE_LENGTH = 64
MAPPING_PATH = "C:/Users/Lenovo/Desktop/New folder/python/Song Generation/mapping.json"
OUTPUT_UNITS = 38
LOSS = "sparse_categorical_crossentropy"
LEARNING_RATE = 0.001
NUM_UNITS = [256]
EPOCHS = 50
BATCH_SIZE = 64
SAVE_MODEL_PATH = "C:/Users/Lenovo/Desktop/New folder/python/Song Generation/model.h5"

In [3]:
def load(file_path):
    with open(file_path, 'r') as f:
        song = f.read()
    return song

In [4]:
def convert_songs_to_int(songs):
    int_songs = []
    
    # load the mappings
    with open(MAPPING_PATH, "r") as f:
        mappings = json.load(f)
    
    # cast song string to a list
    songs = songs.split()

    # map songs to int
    for symbol in songs:
        int_songs.append(mappings[symbol])
    
    return int_songs

In [5]:
def generate_training_sequences(sequence_length):
    # [11, 12, 13, 14, ....], => (input1 = [11, 12] , target1 = [13]) ; (input2 = [12, 13], target2 = [14])
    # network => predict the next item in the time sequence, predict the next musical note given the historical melody
    # LSTM => able to understand memory wise the next event in melody given the historical melody as input
    
    # Load the songs and map them to intergers
    songs = load(SINGLE_FILE_DATASET)
    int_songs = convert_songs_to_int(songs)
    
    inputs = []
    targets = []
    
    # Generate the training sequences
    # sequences length = 64, total symbols = 100 => no of sequences = 100-64 = 36
    num_sequences = len(int_songs) - sequence_length
    for i in range(num_sequences):
        inputs.append(int_songs[i:i+sequence_length])
        targets.append(int_songs[i+sequence_length])
    
    # inputs => (no of sequences, sequence_length)
    
    # One-hot encode the sequences to deal with the categorical data
    # inputs get an extra dimension => inputs => (no of sequences, sequence_length, vocab_size) => 3D array
    vocab_size = len(set(int_songs))
    inputs = keras.utils.to_categorical(inputs, num_classes = vocab_size)
    targets = np.array(targets)
    
    return inputs, targets

In [6]:
def build_model(output_units, num_units, loss, learning_rate):
    
    # create model architecture
    +  
    input = keras.layers.Input(shape=(None, output_units))
    x = keras.layers.LSTM(num_units[0])(input)                   # Passing the input to the LSTM layer using functional API
    x = keras.layers.Dropout(0.2)(x)
    output = keras.layers.Dense(output_units, activation="softmax")(x)
    
    model = keras.Model(input, output)
    
    # compile the model
    model.compile(
        loss=loss,
        optimizer=keras.optimizers.Adam(lr=learning_rate),
        metrics = ['accuracy'])
    
    model.summary()
    
    return model

In [7]:
def train(output_units=OUTPUT_UNITS, num_units=NUM_UNITS, loss=LOSS, learning_rate=LEARNING_RATE):
    
    # generate the training sequences
    inputs, targets = generate_training_sequences(SEQUENCE_LENGTH)
    
    # build the network
    model = build_model(output_units, num_units, loss, learning_rate)
    
    # train the network
    model.fit(inputs, targets, epochs=EPOCHS, batch_size=BATCH_SIZE)
    
    # save the model
    model.save(SAVE_MODEL_PATH)

In [8]:
if __name__=="__main__":
    train()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 38)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               302080    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 38)                9766      
Total params: 311,846
Trainable params: 311,846
Non-trainable params: 0
_________________________________________________________________

Epoch 1/50
362402/362402 [==============================] - 1446s 4ms/step - loss: 0.6832 - accuracy: 0.7885
Epoch 2/50
362402/362402 [==============================] - 578s 2ms/step - loss: 0.5824 - accuracy: 0.8110
Epoch 3/50
362402/362402 [=======